In [2]:
import numpy as np
import redis
import json

In [3]:
redis_host = '10.0.0.5'
redis_pw = "1048576"

In [4]:
r = redis.Redis(host=redis_host, port=6379, db=0, password=redis_pw)

In [12]:
# get all the stored beacon names
beacons_names = r.sscan('BeaconNames', count=r.scard('BeaconNames'))[1]

In [13]:
len(beacons_names)

15

In [25]:
# read/collect all the beacon's readings to this program
all_current_readings = {}

for name in beacons_names:
    all_current_readings[name.decode("utf-8")] = [json.loads(j) for j in r.lrange('BeaconRecord-' + name.decode("utf-8"), 0, 99)]

In [30]:
# extract all BSSID's from all data
all_bssids = {}

for name in all_current_readings:
    beacon_readings = all_current_readings[name]
    # for each reading data point
    for datapt in beacon_readings:
        for ap in datapt['Points']:
            if ap['BSSID'] not in all_bssids:
                all_bssids[ap['BSSID']] = ap['SSID']

In [28]:
all_bssids

{'203a079ef1a0': 'Sinead',
 '34a84e3bb042': 'Giuliano',
 '34a84e3bb043': 'Yuli',
 '34a84e3bb040': 'Sinead'}

In [40]:
# if a value is missing, fill with this value
missing_fill = -101

In [ ]:
readings_list_len = 11

In [36]:
# take the keys list and from it establish the ordering of the aps
# for use in the matrices and vecotrs
matrix_indices = list(all_bssids.keys())

In [88]:
covariant_matrices = {}
mean_vectors = {}

# generate the covariant matrices and mean vectors for the mahalanobis distance
for name in all_current_readings:
    beacon_readings = all_current_readings[name]
    # the table to contain the data
    data_tableu = np.zeros((len(matrix_indices), len(beacon_readings))) + missing_fill
    
    for i in range(len(beacon_readings)):
        datapt = beacon_readings[i]
        for ap in datapt['Points']:
            ind = matrix_indices.index(ap['BSSID'])
            data_tableu[ind, i] = ap['RSSI']
    # remember bias = True in covariance!
    # otherwise the function divides by N-1 instead of N
    cov = np.cov(data_tableu, bias=True)
    # mean vector of the readings
    mean = np.mean(data_tableu, axis=1)
    covariant_matrices[name] = cov
    mean_vectors[name] = mean

In [89]:
covariant_matrices

{'SBU-00': array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]),
 'SBU-12': array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]),
 'SBU-11': array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]),
 'SBU-01': array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]),
 'SBU-09': array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]),
 'SBU-04': array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]),
 'SBU-02': array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]),
 'SBU-06': array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]),
 'SBU-05': array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0

### scratchpad

In [38]:
all_current_readings['SBU-09'][2]['Points']

[{'SSID': 'Sinead', 'BSSID': '203a079ef1a0', 'Channel': 5, 'RSSI': -31},
 {'SSID': 'Giuliano', 'BSSID': '34a84e3bb042', 'Channel': 11, 'RSSI': -56},
 {'SSID': 'Yuli', 'BSSID': '34a84e3bb043', 'Channel': 6, 'RSSI': -66},
 {'SSID': 'Sinead', 'BSSID': '34a84e3bb040', 'Channel': 2, 'RSSI': -81}]

In [60]:
d = [[1.1, -0.43], [1.5, 0.09], [2.1, 0.55], [2.6, 1.21], [2.85, 1.6], [3.3, 1.9]]
e = [[-3.1, 2.4], [-3.1, 2.4], [-3.1, 2.4], [-3.1, 2.4], [-3.1, 2.4], [-3.1, 2.4]]
f = [[i*0.7 - 3, i*1.4+0.1] for i in range(6)]

d = np.array(d)
e = np.array(e)
f = np.array(f)

np.cov(f.T)

array([[1.715, 3.43 ],
       [3.43 , 6.86 ]])

In [73]:
means = np.mean(f.T, axis=1)
means

array([-1.25,  3.6 ])

In [74]:
stds = np.std(f.T, axis=1)
stds

array([1.19547759, 2.39095518])

In [75]:
stds[0]*stds[1]

2.858333333333333

In [82]:
N = 9

f1 = np.array([i*0.7 - 3 for i in range(N)])
f2 = np.array([i*1.4+0.1 for i in range(N)])

f = [f1, f2]

In [83]:
a = np.mean(f1**2) - ( np.mean(f1) )**2
b = np.mean(f1*f2) - np.mean(f1) * np.mean(f2)
c = np.mean(f2**2) - ( np.mean(f2) )**2

In [86]:
mcov = np.array([[a, b], [b, c]])
cov = np.cov(f, bias=True)

In [87]:
cov / mcov

array([[1., 1.],
       [1., 1.]])

In [71]:
g = [[1, 2, 1, 2, 1.5], [1, 1, 2, 2, 1.5]]

np.cov(g)

array([[0.25, 0.  ],
       [0.  , 0.25]])